In [1]:
from nbfinder import NotebookFinder
import sys
import os
sys.meta_path.append(NotebookFinder())
from load_data import get_data
import keras
import tensorflow as tf
from resnet_util import identity_block,conv_block
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

importing Jupyter notebook from load_data.ipynb
importing Jupyter notebook from util.ipynb


Using TensorFlow backend.


importing Jupyter notebook from resnet_util.ipynb


In [2]:
sig_list = []
bg_list = []
sigpath = "/project/projectdirs/dasrepo/icecube_data/hdf5_out/filtered/nugen/11374/clsim-base-4.0.3.0.99_eff/"
for filename in os.listdir(sigpath):
    if filename.endswith("00.hdf5"):
        siglist = os.path.join(sigpath, filename)
        sig_list.append(siglist)
bgpath = "/project/projectdirs/dasrepo/icecube_data/hdf5_out/filtered/corsika/11057/"
for filename in os.listdir(bgpath):
    if filename.endswith("00.hdf5"):
        bglist = os.path.join(bgpath, filename)
        bg_list.append(bglist)
inx,iny = get_data(sig_list, bg_list)

In [3]:
import numpy as np
print inx.shape
print iny.shape

(1388, 60, 10, 20)
(1388,)


In [4]:
inx=np.expand_dims(inx,axis=1)
print inx.shape

(1388, 1, 60, 10, 20)


In [5]:
inx = np.transpose(inx,axes=[0,3,4,2,1])

In [6]:
from keras.layers import *
from keras.models import Model

In [7]:
inputs = Input(shape=inx.shape[1:])

In [8]:
print inputs.shape

(?, 10, 20, 60, 1)


In [9]:
def ResNet50(img_input, classes=2):
    """Instantiates the ResNet50 architecture.
    Optionally loads weights pre-trained
    on ImageNet. Note that when using TensorFlow,
    for best performance you should set
    `image_data_format="channels_last"` in your Keras config
    at ~/.keras/keras.json.
    The model and the weights are compatible with both
    TensorFlow and Theano. The data format
    convention used by the model is the one
    specified in your Keras config file.
    # Arguments
        include_top: whether to include the fully-connected
            layer at the top of the network.
        weights: one of `None` (random initialization)
            or "imagenet" (pre-training on ImageNet).
        input_tensor: optional Keras tensor (i.e. output of `layers.Input()`)
            to use as image input for the model.
        input_shape: optional shape tuple, only to be specified
            if `include_top` is False (otherwise the input shape
            has to be `(224, 224, 3)` (with `channels_last` data format)
            or `(3, 224, 244)` (with `channels_first` data format).
            It should have exactly 3 inputs channels,
            and width and height should be no smaller than 197.
            E.g. `(200, 200, 3)` would be one valid value.
        pooling: Optional pooling mode for feature extraction
            when `include_top` is `False`.
            - `None` means that the output of the model will be
                the 4D tensor output of the
                last convolutional layer.
            - `avg` means that global average pooling
                will be applied to the output of the
                last convolutional layer, and thus
                the output of the model will be a 2D tensor.
            - `max` means that global max pooling will
                be applied.
        classes: optional number of classes to classify images
            into, only to be specified if `include_top` is True, and
            if no `weights` argument is specified.
    # Returns
        A Keras model instance.
    # Raises
        ValueError: in case of invalid argument for `weights`,
            or invalid input shape.
    """

    bn_axis =3
    x = ZeroPadding3D((3, 3, 3))(img_input)
    x = Conv3D(64, (7, 7, 7), strides=(2, 2, 2), name='conv1')(x)
    x = BatchNormalization(axis=bn_axis, name='bn_conv1')(x)
    x = Activation('relu')(x)
    x = MaxPooling3D((3, 3, 3), strides=(2, 2, 2))(x)

    x = conv_block(x, 3, [64, 64, 256], stage=2, block='a', strides=(1, 1, 1))
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='b')
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='c')

    x = conv_block(x, 3, [128, 128, 512], stage=3, block='a', strides=(2, 2, 2))
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='b')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='c')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='d')

    x = conv_block(x, 3, [256, 256, 1024], stage=4, block='a', strides=(2, 2, 2))
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='b')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='c')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='d')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='e')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='f')

    x = conv_block(x, 3, [512, 512, 2048], stage=5, block='a', strides=(2, 2, 2))
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='b')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='c')

    x = AveragePooling3D((1, 1, 1), name='avg_pool')(x)

    x = Flatten()(x)
    x = Dense(classes, activation='softmax', name='fc1000')(x)


    inputs = img_input
    # Create model.
    model = Model(inputs, x, name='resnet50')

    return model



In [10]:
model = ResNet50(img_input=inputs)

In [11]:
from keras.optimizers import adam

In [12]:
opt = adam(lr=0.01)

In [13]:
model.compile(loss="sparse_categorical_crossentropy",optimizer=opt,metrics=["acc"])

In [ ]:
history = model.fit(x=inx,y=iny,validation_split=0.1,epochs=50)

Epoch 1/1
1248/1388 [=========================>....] - ETA: 523s - loss: 2.8301 - acc: 0.8045

In [ ]:
model.save('resnetgpu3d.h5')

In [ ]:
px = model.predict(inx)

In [ ]:
px[px[:,1] == iny].shape

In [ ]:
plt.plot(history.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.savefig('lossresnetgpu3.png')